In [4]:
import pandas as pd, sqlite3, seaborn as sns, matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

#%% TOXCAST BENCHMARK ===========================================================
conn = sqlite3.connect('../../brick/cvae.sqlite')

# get all the property_tokens for tox21 properties
prop_src = pd.read_sql("SELECT property_token,title,source FROM property p INNER JOIN source s on p.source_id = s.source_id", conn)

# pull in multitask_metrics
evaldf = pd.read_csv('../../data/metrics/multitask_metrics.csv')\
    .merge(prop_src, left_on='assay', right_on='property_token', how='inner')

# get the median AUC for each property
evaldf.aggregate({'AUC': 'median','cross_entropy_loss':'median','assay':'count'}) # 89% median auc, .482 median cross entropy loss
evaldf.groupby(['source','nprops']).aggregate({'AUC': 'median','assay':'count'}).sort_values(by='AUC',ascending=False)
evaldf[evaldf['source'] == 'tox21'].groupby(['nprops']).aggregate({'AUC': 'median','assay':'count'}).sort_values(by='AUC',ascending=False)
#              AUC  assay
# nprops                 
# 2       0.909173    110
# 3       0.902808    110
# 4       0.900132    110
# 1       0.871136     66
# 0       0.829932     18

#%% CATEGORICAL BENCHMARK ======================================================

pcat = pd.read_sql("""SELECT property_token,title,category,strength FROM property p 
                       INNER JOIN property_category pc ON p.property_id = pc.property_id
                       INNER JOIN category c on pc.category_id = c.category_id""", 
                       conn)

titles = pcat[pcat['category'] == 'acute inhalation toxicity']
titles = titles[titles['strength'] > 8.0]

# print titles
for i, row in titles[['title']].iterrows():
    print(row['title'])
    
evalcat = pd.read_csv('../../data/metrics/multitask_metrics.csv')\
    .merge(pcat, left_on='assay', right_on='property_token', how='inner')\
    .query('strength > 8.0')

# get the median AUC for each property
evalcat.aggregate({'AUC': 'median','assay':'count'})
pdf = evalcat.groupby(['category','nprops']).aggregate({'AUC': 'median','assay':'count'}).reset_index().sort_values(by=['category','AUC'],ascending=False)
pdf

acute inhalation lc50 testing in mammals
respiratory irritation hazard assessment
inhalation acute toxicity assessment of chemicals
inhalation hazard analysis: h332
measurement of rat acute inhalation toxicity-lc50 values
acute inhalation toxicity assessment of chemicals in ppm
inhalation toxicity assessment


FileNotFoundError: [Errno 2] No such file or directory: 'data/metrics/multitask_metrics.csv'